In [16]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")
from pinecone import Pinecone, ServerlessSpec
import pinecone as pc
from openai import OpenAI
import os
import json
print("OpenAI API Key:", os.getenv("OPENAI_API_KEY"))
print("Pinecone API Key:", os.getenv("PINECONE_API_KEY"))

In [ ]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [2]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Excellent professor! Her lectures are clear, and she's always willing to help."},
 {'professor': 'Prof. Michael Brown',
  'subject': 'Mathematics',
  'stars': 3,
  'review': "Decent teacher, but his lectures can be a bit dry. However, he's very knowledgeable."},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Great professor, very passionate about the subject. Sometimes goes too fast though.'},
 {'professor': 'Prof. John Smith',
  'subject': 'History',
  'stars': 2,
  'review': 'Not very engaging. The class felt more like a reading session than a lecture.'},
 {'professor': 'Dr. Rachel Lee',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Fantastic teacher! She makes complex topics easy to understand.'},
 {'professor': 'Prof. Robert Wilson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Good professor, but sometimes hard to follow. His exams are 

In [3]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-WIRqQ***************************************Dplj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)
print(f"Upserted count: {upsert_response['upserted_count']}")